# Content-based Filtering Analysis

In [144]:
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

## Data Preparation

In [145]:
# Load Data, set configuration variables
item_train = genfromtxt('./Datasets/ContBasData/content_item_train.csv', delimiter=',')
user_train = genfromtxt('./Datasets/ContBasData/content_user_train.csv', delimiter=',')
y_train    = genfromtxt('./Datasets/ContBasData/content_y_train.csv', delimiter=',')
item_vecs = genfromtxt('./Datasets/ContBasData/content_item_vecs.csv', delimiter=',')

movie_dict = defaultdict(dict)
count = 0
#with open('./data/movies.csv', newline='') as csvfile:
with open('./Datasets/ContBasData/content_movie_list.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',', quotechar='"')
    for line in reader:
        if count == 0:
            count += 1  #skip header
            #print(line) print
        else:
            count += 1
            movie_id = int(line[0])
            movie_dict[movie_id]["title"] = line[1]
            movie_dict[movie_id]["genres"] = line[2]
                
num_user_features = user_train.shape[1] - 3  # remove userid, rating count and ave rating during training
num_item_features = item_train.shape[1] - 1  # remove movie id at train time
uvs = 3  # user genre vector start
ivs = 3  # item genre vector start
u_s = 3  # start of columns to use in training, user
i_s = 1  # start of columns to use in training, items
print(f"Number of training vectors: {len(item_train)}")

Number of training vectors: 50884


- item_train consists of movie id, year, average rating, then 14 genres each as a column following the one-hot encoding process which uses binary classification to describe the genre of each movie. 
- user_train consists of user id, rating count, rating average, then the 0 to 5 scale on each genre, describing the preference of the user.
- y_train is the movie rating given by the user.

In [131]:
# scale training data
item_train_unscaled = item_train
user_train_unscaled = user_train
y_train_unscaled    = y_train

scalerItem = StandardScaler()
scalerItem.fit(item_train)
item_train = scalerItem.transform(item_train)

scalerUser = StandardScaler()
scalerUser.fit(user_train)
user_train = scalerUser.transform(user_train)

scalerTarget = MinMaxScaler((-1, 1))
scalerTarget.fit(y_train.reshape(-1, 1))
y_train = scalerTarget.transform(y_train.reshape(-1, 1))
#ynorm_test = scalerTarget.transform(y_test.reshape(-1, 1))

print(np.allclose(item_train_unscaled, scalerItem.inverse_transform(item_train)))
print(np.allclose(user_train_unscaled, scalerUser.inverse_transform(user_train)))

True
True


In [132]:
# splitting data into train and test sets
item_train, item_test = train_test_split(item_train, train_size=0.80, shuffle=True, random_state=1)
user_train, user_test = train_test_split(user_train, train_size=0.80, shuffle=True, random_state=1)
y_train, y_test       = train_test_split(y_train,    train_size=0.80, shuffle=True, random_state=1)
print(f"movie/item training data shape: {item_train.shape}")
print(f"movie/item test data shape: {item_test.shape}")

movie/item training data shape: (40707, 17)
movie/item test data shape: (10177, 17)


## Building Neural Network Model for content-based filtering

In [133]:
num_outputs = 32
tf.random.set_seed(1)
user_NN = tf.keras.models.Sequential([ 
    tf.keras.layers.Dense(units = 256, activation = 'relu'),
    tf.keras.layers.Dense(units = 128, activation = 'relu'),
    tf.keras.layers.Dense(units = num_outputs, activation = 'linear') 
])

item_NN = tf.keras.models.Sequential([     
    tf.keras.layers.Dense(units = 256, activation = 'relu'),
    tf.keras.layers.Dense(units = 128, activation = 'relu'),
    tf.keras.layers.Dense(units = num_outputs, activation = 'linear') 
])

# create the user input and point to the base network
input_user = tf.keras.layers.Input(shape=(num_user_features))
vu = user_NN(input_user)
vu = tf.linalg.l2_normalize(vu, axis=1)

# create the item input and point to the base network
input_item = tf.keras.layers.Input(shape=(num_item_features))
vm = item_NN(input_item)
vm = tf.linalg.l2_normalize(vm, axis=1)

# compute the dot product of the two vectors vu and vm
output = tf.keras.layers.Dot(axes=1)([vu, vm])

# specify the inputs and output of the model
model = tf.keras.Model([input_user, input_item], output)

model.summary()

Model: "model_17"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_35 (InputLayer)          [(None, 14)]         0           []                               
                                                                                                  
 input_36 (InputLayer)          [(None, 16)]         0           []                               
                                                                                                  
 sequential_36 (Sequential)     (None, 32)           40864       ['input_35[0][0]']               
                                                                                                  
 sequential_37 (Sequential)     (None, 32)           41376       ['input_36[0][0]']               
                                                                                           

In [134]:
#use MSE loss and Adam optimizer
tf.random.set_seed(1)
model.compile(optimizer=keras.optimizers.legacy.Adam(learning_rate=0.01),
              loss=tf.keras.losses.MeanSquaredError())

In [135]:
tf.random.set_seed(1)
model.fit([user_train[:, u_s:], item_train[:, i_s:]], y_train, epochs=30)

Epoch 1/30
1273/1273 [==============================] - 1s 614us/step - loss: 0.1230
Epoch 2/30
1273/1273 [==============================] - 1s 614us/step - loss: 0.1136
Epoch 3/30
1273/1273 [==============================] - 1s 615us/step - loss: 0.1085
Epoch 4/30
1273/1273 [==============================] - 1s 614us/step - loss: 0.1039
Epoch 5/30
1273/1273 [==============================] - 1s 623us/step - loss: 0.1016
Epoch 6/30
1273/1273 [==============================] - 1s 620us/step - loss: 0.0987
Epoch 7/30
1273/1273 [==============================] - 1s 616us/step - loss: 0.0964
Epoch 8/30
1273/1273 [==============================] - 1s 644us/step - loss: 0.0945
Epoch 9/30
1273/1273 [==============================] - 1s 615us/step - loss: 0.0927
Epoch 10/30
1273/1273 [==============================] - 1s 615us/step - loss: 0.0913
Epoch 11/30
1273/1273 [==============================] - 1s 613us/step - loss: 0.0897
Epoch 12/30
1273/1273 [==============================] - 1s 614

In [136]:
# evaluate the model on test data
model.evaluate([user_test[:, u_s:], item_test[:, i_s:]], y_test)

319/319 [==============================] - 0s 272us/step - loss: 0.0822


0.08220424503087997

The loss for the test set is not too different from that of the training set. Hence the model is not overfitting the data.
## Making Predictions

In [137]:
# creating new user and predicting movies
new_user_id = 5000
new_rating_ave = 0.0
new_action = 0.0
new_adventure = 5.0
new_animation = 0.0
new_childrens = 0.0
new_comedy = 0.0
new_crime = 0.0
new_documentary = 0.0
new_drama = 0.0
new_fantasy = 5.0
new_horror = 0.0
new_mystery = 0.0
new_romance = 0.0
new_scifi = 0.0
new_thriller = 0.0
new_rating_count = 3

user_vec = np.array([[new_user_id, new_rating_count, new_rating_ave,
                      new_action, new_adventure, new_animation, new_childrens,
                      new_comedy, new_crime, new_documentary,
                      new_drama, new_fantasy, new_horror, new_mystery,
                      new_romance, new_scifi, new_thriller]])

In [141]:
# useful functions

# return user predict matrix to match the size of item_vecs, given user_vec
def gen_user_vecs(user_vec, num_items):
    user_vecs = np.tile(user_vec, (num_items, 1))
    return user_vecs

#given a user_id, return: user train/predict matrix to match the size of item_vecs y vector with ratings for all rated movies and 0 for others of size item_vecs
def get_user_vecs(user_id, user_train, item_vecs, user_to_genre):
    if not user_id in user_to_genre:
        print("error: unknown user id")
        return None
    else:
        user_vec_found = False
        for i in range(len(user_train)):
            if user_train[i, 0] == user_id:
                user_vec = user_train[i]
                user_vec_found = True
                break
        if not user_vec_found:
            print("error in get_user_vecs, did not find uid in user_train")
        num_items = len(item_vecs)
        user_vecs = np.tile(user_vec, (num_items, 1))

        y = np.zeros(num_items)
        for i in range(num_items):  # walk through movies in item_vecs and get the movies, see if user has rated them
            movie_id = item_vecs[i, 0]
            if movie_id in user_to_genre[user_id]['movies']:
                rating = user_to_genre[user_id]['movies'][movie_id]
            else:
                rating = 0
            y[i] = rating
    return(user_vecs, y)

#print results of prediction of a new user. inputs are expected to be in sorted order, unscaled.
def print_pred_movies(y_p, item, movie_dict, maxcount=10):
    count = 0
    disp = [["y_p", "movie id", "rating ave", "title", "genres"]]

    for i in range(0, y_p.shape[0]):
        if count == maxcount:
            break
        count += 1
        movie_id = item[i, 0].astype(int)
        disp.append([np.around(y_p[i, 0], 1), item[i, 0].astype(int), np.around(item[i, 2].astype(float), 1),
                     movie_dict[movie_id]['title'], movie_dict[movie_id]['genres']])

    table = tabulate.tabulate(disp, tablefmt='html', headers="firstrow")
    return table

In [139]:
from collections import defaultdict
import csv
import tabulate

# generate and replicate the user vector to match the number movies in the data set.
item_vecs = genfromtxt('./Datasets/ContBasData/content_item_vecs.csv', delimiter=',')
user_vecs = gen_user_vecs(user_vec,len(item_vecs))
movie_dict = defaultdict(dict)
count = 0
#with open('./data/movies.csv', newline='') as csvfile:
with open('./Datasets/COntBasData/content_movie_list.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',', quotechar='"')
    for line in reader:
        if count == 0:
            count += 1  #skip header
            #print(line) print
        else:
            count += 1
            movie_id = int(line[0])
            movie_dict[movie_id]["title"] = line[1]
            movie_dict[movie_id]["genres"] = line[2]

# scale our user and item vectors
suser_vecs = scalerUser.transform(user_vecs)
sitem_vecs = scalerItem.transform(item_vecs)

# make a prediction
y_p = model.predict([suser_vecs[:, u_s:], sitem_vecs[:, i_s:]])

# unscale y prediction 
y_pu = scalerTarget.inverse_transform(y_p)

# sort the results, highest prediction first
sorted_index = np.argsort(-y_pu,axis=0).reshape(-1).tolist()  #negate to get largest rating first
sorted_ypu   = y_pu[sorted_index]
sorted_items = item_vecs[sorted_index]  #using unscaled vectors for display

print_pred_movies(sorted_ypu, sorted_items, movie_dict, maxcount = 10)

27/27 [==============================] - 0s 362us/step


y_p,movie id,rating ave,title,genres
4.4,8368,3.9,Harry Potter and the Prisoner of Azkaban (2004),Adventure|Fantasy
4.4,54001,3.9,Harry Potter and the Order of the Phoenix (2007),Adventure|Drama|Fantasy
4.4,81834,4,Harry Potter and the Deathly Hallows: Part 1 (2010),Action|Adventure|Fantasy
4.3,137857,3.6,The Jungle Book (2016),Adventure|Drama|Fantasy
4.3,59387,4,"Fall, The (2006)",Adventure|Drama|Fantasy
4.3,5952,4,"Lord of the Rings: The Two Towers, The (2002)",Adventure|Fantasy
4.3,98809,3.8,"Hobbit: An Unexpected Journey, The (2012)",Adventure|Fantasy
4.3,5816,3.6,Harry Potter and the Chamber of Secrets (2002),Adventure|Fantasy
4.3,40815,3.8,Harry Potter and the Goblet of Fire (2005),Adventure|Fantasy|Thriller
4.2,7153,4.1,"Lord of the Rings: The Return of the King, The (2003)",Action|Adventure|Drama|Fantasy


## Finding similar items

In [146]:
# Returns the squared distance between two vectors
def sq_dist(a,b):
    """
    Args:
      a (ndarray (n,)): vector with n features
      b (ndarray (n,)): vector with n features
    Returns:
      d (float) : distance
    """
    ### START CODE HERE ###     
    d = np.sum(np.square(a-b))
    ### END CODE HERE ###     
    return d

In [147]:
input_item_m = tf.keras.layers.Input(shape=(num_item_features))    # input layer
vm_m = item_NN(input_item_m)                                       # use the trained item_NN
vm_m = tf.linalg.l2_normalize(vm_m, axis=1)                        # incorporate normalization as was done in the original model
model_m = tf.keras.Model(input_item_m, vm_m)                                
model_m.summary()

Model: "model_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_37 (InputLayer)       [(None, 16)]              0         
                                                                 
 sequential_37 (Sequential)  (None, 32)                41376     
                                                                 
 tf.math.l2_normalize_34 (TF  (None, 32)               0         
 OpLambda)                                                       
                                                                 
Total params: 41,376
Trainable params: 41,376
Non-trainable params: 0
_________________________________________________________________


In [148]:
scaled_item_vecs = scalerItem.transform(item_vecs)
vms = model_m.predict(scaled_item_vecs[:,i_s:])
print(f"size of all predicted movie feature vectors: {vms.shape}")

27/27 [==============================] - 0s 352us/step
size of all predicted movie feature vectors: (847, 32)


In [151]:
count = 50  # number of movies to display
dim = len(vms)
dist = np.zeros((dim,dim))

for i in range(dim):
    for j in range(dim):
        dist[i,j] = sq_dist(vms[i, :], vms[j, :])
        
m_dist = np.ma.masked_array(dist, mask=np.identity(dist.shape[0]))  # mask the diagonal

disp = [["movie1", "genres", "movie2", "genres"]]
for i in range(count):
    min_idx = np.argmin(m_dist[i])
    movie1_id = int(item_vecs[i,0])
    movie2_id = int(item_vecs[min_idx,0])
    disp.append( [movie_dict[movie1_id]['title'], movie_dict[movie1_id]['genres'],
                  movie_dict[movie2_id]['title'], movie_dict[movie1_id]['genres']]
               )
table = tabulate.tabulate(disp, tablefmt='html', headers="firstrow")
table

movie1,genres,movie2,genres
Save the Last Dance (2001),Drama|Romance,Mona Lisa Smile (2003),Drama|Romance
"Wedding Planner, The (2001)",Comedy|Romance,Mr. Deeds (2002),Comedy|Romance
Hannibal (2001),Horror|Thriller,Final Destination 2 (2003),Horror|Thriller
Saving Silverman (Evil Woman) (2001),Comedy|Romance,"Sweetest Thing, The (2002)",Comedy|Romance
Down to Earth (2001),Comedy|Fantasy|Romance,Bewitched (2005),Comedy|Fantasy|Romance
"Mexican, The (2001)",Action|Comedy,Rush Hour 2 (2001),Action|Comedy
15 Minutes (2001),Thriller,Panic Room (2002),Thriller
Enemy at the Gates (2001),Drama,Seabiscuit (2003),Drama
Heartbreakers (2001),Comedy|Crime|Romance,Fun with Dick and Jane (2005),Comedy|Crime|Romance
Spy Kids (2001),Action|Adventure|Children|Comedy,Scooby-Doo (2002),Action|Adventure|Children|Comedy
